# 1. ThreadPoolExecutor

Pula wątków przeważnie jest używana w zadaniach zależnych od I/O (I/O bounded tasks).

TODO: add more description


# Przykład: scraper Wikipedii

Scenariusz: dla danej listy haseł (`TERMS`) pobierz zawartość pierwszych paragrafów artykułów Wikipedii dla każdego hasła.

Dane:
 - `WIKI_URL`: bazowy adres angielskiej Wikipedii. W każdym zadaniu hasło jest dołączane na koniec tego adresu (np. `'https://en.wikipedia.org/wiki/' + 'kite'` daje [adres do artykułu o latawcu](https://en.wikipedia.org/wiki/kite)).
 - `TERMS`: lista haseł do wyszukania.
 - `get_from_wiki`: funkcja, która odpytuje adres dla danego hasła, sprawdza kod odpowiedzi i zwraca krotkę kodu statusu i tekst pierwszego paragrafu danego artykułu Wikipedii.
 - `get_first_paragraph`: funkcja pomocnicza parsująca zawartość HTML i wyciągająca tekst pierwszego paragrafu.
 - `timeit`: funkcja dekorująca, mierząca czas wykonania dekorowanej funkcji.

In [ ]:
from lxml import html
import time
from typing import *

import requests

WIKI_URL = 'https://en.wikipedia.org/wiki/'
TERMS = [
    'family',
    'measurement',
    'leader',
    'atmosphere',
    'possibility',
    'housing',
    'payment',
    'sympathy',
    'meal',
    'description',
    'intention',
    'community',
    'preference',
    'menu',
    'volume',
    'brewery',
    'abcdefgh',  # no article
    'assumption',
    'patience',
    'recipe',
]


def timeit(func):
    """Wraps the function for measuring its execution time."""
    
    def wrapped(*args, **kwargs):
        t_start = time.time()
        result = func(*args, **kwargs)
        print(f'Executed `{func.__name__}` in {(time.time() - t_start):.2f}s')
        return result
    
    return wrapped


def get_first_paragraph(html_text: str) -> str:
    """
    Returns a text from first paragraph of given html content.
    """
    tree = html.fromstring(html_text)
    paragraph = tree.find('body//p')
    if isinstance(paragraph, html.HtmlElement):
        return paragraph.text_content().strip()
    return ''
    

def get_from_wiki(term: str) -> Tuple[int, str]:
    """
    Returns the status code and text of first paragraph
    from wikipedia article in form of a tuple.
    """
    res = requests.get(WIKI_URL + term)
    status = res.status_code
    if res.status_code != 200:
        return status, ''
    return status, get_first_paragraph(res.content)



### Przykładowy wynik dla [artykułu o data scrapinu](https://en.wikipedia.org/wiki/Data_scraping), wywołując funkcję  `get_from_wiki`:


In [ ]:
code, text = get_from_wiki('data_scraping')
print(f'response code: {code}')
print(f'text: {text}')



## Typowe zadanie sekwencyjne, które wywołuje funkcję dla każdego hasła


In [ ]:
@timeit
def task_sequential(terms):
    return [get_from_wiki(term) for term in terms]



#### Test `task_sequential`:


In [ ]:
result_sequential = task_sequential(TERMS)

for term, (code, text) in zip(TERMS, result_sequential):
    print(f'{term}, response code: {code}')
    print(text, '\n')


## Zrównoleglone zadanie: podejście 1.
Zrównoleglone zadanie z użyciem klasy `ThreadPoolExecutor` i metody `submit()`. Zwraca listę wyników.

In [ ]:
from concurrent.futures import ThreadPoolExecutor


@timeit
def task_parallel(terms, n_workers=10):
    with ThreadPoolExecutor(n_workers) as pool:
        futures = [pool.submit(get_from_wiki, term) for term in terms]
    return [future.result() for future in futures]

#### Test `task_parallel`:


In [ ]:
result_parallel = task_parallel(TERMS)
result_sequential == result_parallel  # same result?


## Zrównoleglone zadanie: podejście 2.
Zrównoleglone zadanie z użyciem klasy `ThreadPoolExecutor` i metody `map()`. Zwraca listę wyników.

In [ ]:
@timeit
def task_parallel_2(terms, n_workers=10):
    with ThreadPoolExecutor(n_workers) as pool:
        result = pool.map(get_from_wiki, terms)
    return list(result)

In [ ]:
longer_list = TERMS * 3
_ = task_parallel_2(longer_list)
_ = task_parallel_2(longer_list, n_workers=30)

## Zrównoleglone zadanie: podejście 3.

Zrównoleglone zadanie z użyciem klasy `ThreadPoolExecutor` i metody `map()`. Leniwe podejście.

In [ ]:
@timeit
def task_parallel_3(terms, n_workers=10):
    with ThreadPoolExecutor(n_workers) as pool:
        yield from pool.map(get_from_wiki, terms)

In [ ]:
res = task_parallel_3(TERMS, n_workers=2)
res
time.sleep(1)
for code, text in res:
    print(code)